<a href="https://colab.research.google.com/github/amitvapal/rag-price-prediction/blob/main/claude_LLM_PricePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install PyTorch and transformers
!pip install torch transformers accelerate huggingface_hub bitsandbytes anthropic

# Install RAPIDS libraries for GPU
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com

# Install ChromaDB and Items
!pip install -q chromadb Items

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 38.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 184.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 MB 192.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cuda-nvrtc-cu12
    Found existing installation: nvidia-cuda-nvrtc-cu12 12.6.77
    Uninstalling nvidia-cuda-nvrtc-cu12-12.6.77:
      Successfully uninstalled nvidia-cuda-nvrtc-cu12-12.6.77
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [6]:
import torch

# Check CUDA version
!nvcc --version

print("="*70)
print("GPU VERIFICATION")
print("="*70)
print(f"CUDA Available: {torch.cuda.is_available()}")
assert torch.cuda.is_available(), "❌ CUDA GPU not found! Change runtime to GPU."

print(f"GPU Device: {torch.cuda.get_device_name(0)}")
print(f"Current Device: {torch.cuda.current_device()}")
print(f"Device Count: {torch.cuda.device_count()}")

gpu_name = torch.cuda.get_device_name(0)
if "L4" not in gpu_name:
    print(f"⚠️ Non-L4 GPU detected: {gpu_name}")
else:
    print(f"✓ Using L4 GPU: {gpu_name}")
print("="*70)

# Set device
device = torch.device('cuda')

ImportError: /usr/local/lib/python3.12/dist-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: undefined symbol: __nvJitLinkGetErrorLogSize_12_9, version libnvJitLink.so.12

In [ ]:
from google.colab import userdata
import anthropic

# Get API key from Colab secrets
api_key = userdata.get('ANTHROPIC_API_KEY')

# Create Claude client
claude_client = anthropic.Anthropic(api_key=api_key)

print("✓ Claude API client initialized")


✓ Claude API client initialized


In [ ]:
from google.colab import userdata
from huggingface_hub import login

# Get the token from Colab's secret manager
hf_token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token=hf_token)

print("✓ Successfully logged in to Hugging Face!")

✓ Successfully logged in to Hugging Face!


In [ ]:
from google.colab import drive
import os

# Unmount if already mounted
drive.flush_and_unmount()

# Remount
drive.mount('/content/drive', force_remount=True)


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
# Navigate to your folder
os.chdir('/content/drive/MyDrive/claude-LLM-PricePredictor')

# Get current working directory
print(f"Current directory: {os.getcwd()}")

# List contents
print("\nContents:")
for item in os.listdir('.'):
    print(f"  📁 {item}" if os.path.isdir(item) else f"  📄 {item}")

# Full path example
file_path = os.path.join(os.getcwd(), 'redoing-205_week8_day2_Claude_predict_price_RAG_pipeline.ipynb')
print(f"Full path: {file_path}")

Current directory: /content/drive/MyDrive/claude-LLM-PricePredictor

Contents:
  📄 claude-LLM-PricePredictor.ipynb
  📁 data
  📁 scripts
  📁 products_vetorstore
Full path: /content/drive/MyDrive/claude-LLM-PricePredictor/redoing-205_week8_day2_Claude_predict_price_RAG_pipeline.ipynb


In [ ]:
# ===================================================================
# STEP 1: Standard library imports
# ===================================================================
import sys
import os
import re
import math
import json
import random
from datetime import datetime
import pickle
import time

# ===================================================================
# STEP 2: Third-party imports
# ===================================================================
from tqdm import tqdm
from dotenv import load_dotenv

import torch
# from sentence_transformers import SentenceTransformer # note

# # Note: SentenceTransformer Library: This library receives your request. It says, "Okay, I need to build an embedding model using meta-llama/Meta-Llama-3.1-8B as the foundation."

# # transformers Library (Under the hood): SentenceTransformer uses the transformers library to do the actual downloading. It makes a request to the Hugging Face Hub.



import chromadb

from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go

# ===================================================================
# STEP 3: Setup Python path BEFORE importing custom modules
# ===================================================================
base_path = '/content/drive/MyDrive/claude-LLM-PricePredictor'

# Change to base directory
os.chdir(base_path)

# Add scripts folder to Python path
scripts_path = os.path.join(base_path, 'scripts')
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

print("="*70)
print("PATH SETUP")
print("="*70)
print(f"Current directory: {os.getcwd()}")
print(f"Scripts path added: {scripts_path}")
print(f"Scripts folder exists: {os.path.exists(scripts_path)}")

# List files in scripts folder to verify
if os.path.exists(scripts_path):
    print("\nFiles in scripts folder:")
    for file in os.listdir(scripts_path):
        print(f"  - {file}")

print("="*70)

# ===================================================================
# STEP 4: NOW import custom modules (after path is configured)
# ===================================================================
try:
    from preprocess_data import Item
    print("✓ Successfully imported Item from preprocess_data")
except ImportError as e:
    print(f"❌ Failed to import Item: {e}")
    # Try alternative import
    try:
        from items import Item
        print("✓ Successfully imported Item from items")
    except ImportError as e2:
        print(f"❌ Failed to import from items too: {e2}")

try:
    from testing import Tester
    print("✓ Successfully imported Tester from testing")
except ImportError as e:
    print(f"❌ Failed to import Tester: {e}")

print("\n✓ All imports configured successfully")

PATH SETUP
Current directory: /content/drive/MyDrive/claude-LLM-PricePredictor
Scripts path added: /content/drive/MyDrive/claude-LLM-PricePredictor/scripts
Scripts folder exists: True

Files in scripts folder:
  - preprocess_data.py
  - items.py
  - testing.py
  - __init__.py
  - loaders.py
  - loaders_for_colab.py
  - loaders_for_colab_version_two.py


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

✓ Successfully imported Item from preprocess_data
✓ Successfully imported Tester from testing

✓ All imports configured successfully


In [ ]:
## 11. **Helper Functions**

def description(item):
    """Extract clean product description from item"""
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split('\n\nPrice is $')[0]


def vector(item):
    """Generate embedding vector for item using Sentence Transformer (GPU optimized)"""
    with torch.no_grad():
        embedding = model_sentence_transformer.encode(
            [description(item)],
            convert_to_tensor=True,
            device='cuda'
        )
    return embedding.cpu().numpy()  # Convert to numpy for ChromaDB


def find_similars(item, n_results=5):
    """Find similar items from ChromaDB vector store"""
    results = collection.query(
        query_embeddings=vector(item).astype(float).tolist(),
        n_results=n_results
    )
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices


def make_context(similars, prices):
    """Format similar items into context string for LLM"""
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message


def get_price(s):
    """Extract numeric price from string"""
    s = s.replace('$', '').replace(',', '')
    match = re.search(r"[-+]?\d*\.?\d+", s)
    return float(match.group()) if match else 0


print("✓ All helper functions defined")

✓ All helper functions defined


In [ ]:
print("="*70)
print("COPYING DATASET TO LOCAL VM")
print("="*70)

# Define paths
drive_data_folder = "/content/drive/MyDrive/claude-LLM-PricePredictor/data"
local_data_folder = "/content/final_data"

# Copy from Drive to local disk for faster access
print(f"Copying from Drive to {local_data_folder}...")
!cp -r "$drive_data_folder" "$local_data_folder"
print("✓ Copy complete")

# Update data_folder variable
data_folder = local_data_folder

# Load pickle files
print("Loading pickle files...")
with open(os.path.join(data_folder, "train.pkl"), 'rb') as file:
    train = pickle.load(file)

with open(os.path.join(data_folder, "test.pkl"), 'rb') as file:
    test = pickle.load(file)

print(f"✓ Loaded train: {len(train):,} items")
print(f"✓ Loaded test: {len(test):,} items")
print("="*70)

COPYING DATASET TO LOCAL VM
Copying from Drive to /content/final_data...
✓ Copy complete
Loading pickle files...
✓ Loaded train: 400,000 items
✓ Loaded test: 26,934 items


## 8. **Checking Train data**

In [ ]:
# Loop through the first three items in the train list
for i in range(2):
    print(f"========== ITEM {i} ==========")

    # Get all attributes of the item as a dictionary
    item_attributes = vars(train[i])

    # Loop through the dictionary and print each key-value pair
    for key, value in item_attributes.items():
        print(f"{key}: {value}")

    print("\n") # Add a newline to separate the items

========== ITEM 0 ==========
title: Front Strut Coil Spring Sway Bar Link Kit - 4 Piece - Compatible with 2010-2016 Cadillac SRX (without RPO Code F45)
price: 268.95
details: {"Manufacturer": "Marketplace Auto Parts", "Brand": "Marketplace Auto Parts", "Item model number": "W0129-F619516", "Manufacturer Part Number": "W0129-F619516", "Best Sellers Rank": {"Automotive": 1510741, "Automotive Replacement Struts": 9567}, "Date First Available": "February 5, 2021"}
prompt: How much does this cost to the nearest dollar?

Front Strut Coil Spring Sway Bar Link Kit - 4 Piece - Compatible with 2010-2016 Cadillac SRX (without RPO Code F45)
Front Strut Coil Spring Sway Bar Link Kit 4 Piece Set - Compatible with 2010 - 2016 Cadillac SRXPosition  FrontNote  Front; without RPO Code F45  Real Time Damping Chassis ; Except RPO Code F45  Real Time Damping Chassis Quantity  4 Piece SetPackage Contents  Sway Bar LinksFeatures   - Matched Parts - Pre-AssembledCompatible With or Fits   - 2010 - 2016 Cadilla

In [ ]:
# Loop through the first three items
for i in range(3):
    print(f"========== ATTRIBUTES FOR ITEM {i} ==========")

    # Get the dictionary of attributes and then get just the keys
    attribute_names = vars(train[i]).keys()

    # Print the list of attribute names
    print(list(attribute_names))
    print("\n")

========== ATTRIBUTES FOR ITEM 0 ==========
['title', 'price', 'details', 'prompt', 'token_count', 'include', 'category']


========== ATTRIBUTES FOR ITEM 1 ==========
['title', 'price', 'details', 'prompt', 'token_count', 'include', 'category']


========== ATTRIBUTES FOR ITEM 2 ==========
['title', 'price', 'details', 'prompt', 'token_count', 'include', 'category']




In [ ]:
len(train)

400000

In [ ]:
train[0]

<Front Strut Coil Spring Sway Bar Link Kit - 4 Piece - Compatible with 2010-2016 Cadillac SRX (without RPO Code F45) = $268.95>

In [ ]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nFront Strut Coil Spring Sway Bar Link Kit - 4 Piece - Compatible with 2010-2016 Cadillac SRX (without RPO Code F45)\nFront Strut Coil Spring Sway Bar Link Kit 4 Piece Set - Compatible with 2010 - 2016 Cadillac SRXPosition  FrontNote  Front; without RPO Code F45  Real Time Damping Chassis ; Except RPO Code F45  Real Time Damping Chassis Quantity  4 Piece SetPackage Contents  Sway Bar LinksFeatures   - Matched Parts - Pre-AssembledCompatible With or Fits   - 2010 - 2016 Cadillac SRX Front; without RPO Code F45  Real Time Damping Chassis ; Except RPO Code F45  Real Time\n\nPrice is $269.00'

In [ ]:
description(train[0])

'Front Strut Coil Spring Sway Bar Link Kit - 4 Piece - Compatible with 2010-2016 Cadillac SRX (without RPO Code F45)\nFront Strut Coil Spring Sway Bar Link Kit 4 Piece Set - Compatible with 2010 - 2016 Cadillac SRXPosition  FrontNote  Front; without RPO Code F45  Real Time Damping Chassis ; Except RPO Code F45  Real Time Damping Chassis Quantity  4 Piece SetPackage Contents  Sway Bar LinksFeatures   - Matched Parts - Pre-AssembledCompatible With or Fits   - 2010 - 2016 Cadillac SRX Front; without RPO Code F45  Real Time Damping Chassis ; Except RPO Code F45  Real Time'

In [ ]:
# Loop through the first three items in the train list
for i in range(2):
    print(f"========== ITEM {i} ==========")

    # Get all attributes of the item as a dictionary
    item_attributes = vars(test[i])

    # Loop through the dictionary and print each key-value pair
    for key, value in item_attributes.items():
        print(f"{key}: {value}")

    print("\n") # Add a newline to separate the items

========== ITEM 0 ==========
title: Sony Dt 55-200mm F4-5.6 SAM SAL55200-2 - International Version (No Warranty)
price: 82.99
details: {"Product Dimensions": "9 x 7 x 5 inches", "Item Weight": "10.8 ounces", "Item model number": "SAL55200-2", "Is Discontinued By Manufacturer": "No", "Date First Available": "September 13, 2013", "Manufacturer": "Sony", "Brand": "Sony", "Focal Length Description": "55-200 millimeters", "Lens Type": "Telephoto", "Compatible Mountings": "Sony A-mount", "Camera Lens Description": "200 month"}
prompt: How much does this cost to the nearest dollar?

Sony Dt 55-200mm F4-5.6 SAM SAL55200-2 - International Version (No Warranty)
This is an international version  an international version item is a product not originally intended for sale in the U.S. Depending on the country of origin,, international versions of a product may differ from domestic versions in their default settings,, packaging,, AC adapters,, accessories,, Manufacturer warranties,, and other feature

In [ ]:

import chromadb
import os
from sentence_transformers import SentenceTransformer
# Use the existing folder path from your Drive
DB = "products_vectorstore"
# db_path = "/content/drive/MyDrive/01-LLM-ed-donner/build_multi_agent_system/test_products_vectorstore"

# Use the Colab content directory for fast local storage
db_path = f"/content/{DB}"

# Create client
# Initializes the ChromaDB client. The use of PersistentClient means that all data added to the database will be saved to the specified db_path folder.
client = chromadb.PersistentClient(path=db_path)


# Check and delete collection if exists

# Sets the name of the vector collection i.e. "products"(similar to a table in a traditional database) that will store your product vectors.
collection_name = "products"

existing_collection_names = [collection.name for collection in client.list_collections()]


# Checks if the "products" collection is in the list of existing collections.If the collection "products" exists, it is permanently deleted.
if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted old collection: {collection_name}")

# Create collection
# Creates a new, empty vector collection named "products" and assigns the resulting collection object to the collection variable. This object is what you use to add, query, or delete vectors.
collection = client.create_collection(collection_name)
print(f"✓ Collection '{collection_name}' created at {db_path}")



Deleted old collection: products
✓ Collection 'products' created at /content/products_vectorstore


In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

# Model is already on GPU (confirmed above)
device = torch.device('cuda')



# SentenceTransformer is a Python framework that uses pre-trained Transformer models (like BERT, RoBERTa, or MiniLM) to generate high-quality sentence and text embeddings. It converts pieces of text (sentences, paragraphs, or documents) into dense numerical vectors (embeddings) that capture their semantic meaning. Texts with similar meanings will have vectors that are close together in this high-dimensional space.
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

print("Starting optimized vectorization...")


# Process in larger batches with better GPU utilization
# Recommendation: Since we are using a high-end GPU (or if plan to use one), try to increase this inner batch size (e.g., 512, 1024, or even 2048) until you encounter a CUDA out-of-memory (OOM) error. A larger batch size keeps the GPU cores saturated with work.
batch_size = 2048
encoding_batch_size = 2048 # How many to encode at once on GPU

for i in tqdm(range(0, len(train), batch_size)):
    batch_end = min(i + batch_size, len(train))
    batch_items = train[i:batch_end]

    # Extract all data at once. Note: description function above
    documents = [description(item) for item in batch_items]

    # GPU-optimized encoding with larger internal batches
    vectors = model.encode(
        documents,
        batch_size=encoding_batch_size,
        show_progress_bar=False,
        convert_to_numpy=True,
        normalize_embeddings=False,
        device=device
    ).tolist()

    # Prepare metadata
    metadatas = [{"category": item.category, "price": item.price} for item in batch_items]
    ids = [f"doc_{j}" for j in range(i, batch_end)]

    # Single ChromaDB insertion
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

print(f"\n✓ Vectorization complete!")

Starting optimized vectorization...


100%|██████████| 196/196 [12:58<00:00,  3.97s/it]


✓ Vectorization complete!


## 10. **Load Sentence Transformer Model**

In [ ]:
print("="*70)
print("LOADING SENTENCE TRANSFORMER ON GPU")
print("="*70)

# Force GPU usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Loading Sentence Transformer on {device}...")

model_sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2', device=device)

print(f"✓ Sentence Transformer on: {model_sentence_transformer.device}")
print("="*70)

LOADING SENTENCE TRANSFORMER ON GPU
Loading Sentence Transformer on cuda...
✓ Sentence Transformer on: cuda:0


## 11. **Inspect: Checking Chroma DB Vector Database**

In [ ]:
# total number of items
total_items = collection.count()

print(f"Total items in 'products' collection: {total_items}")

Total items in 'products' collection: 400000


In [ ]:
# quick look to data (IDs, documents, and metadata).
import pprint
pprint.pprint(collection.peek(limit=1))

{'data': None,
 'documents': ['Front Strut Coil Spring Sway Bar Link Kit - 4 Piece - '
               'Compatible with 2010-2016 Cadillac SRX (without RPO Code F45)\n'
               'Front Strut Coil Spring Sway Bar Link Kit 4 Piece Set - '
               'Compatible with 2010 - 2016 Cadillac SRXPosition  FrontNote  '
               'Front; without RPO Code F45  Real Time Damping Chassis ; '
               'Except RPO Code F45  Real Time Damping Chassis Quantity  4 '
               'Piece SetPackage Contents  Sway Bar LinksFeatures   - Matched '
               'Parts - Pre-AssembledCompatible With or Fits   - 2010 - 2016 '
               'Cadillac SRX Front; without RPO Code F45  Real Time Damping '
               'Chassis ; Except RPO Code F45  Real Time'],
 'embeddings': array([[-8.05460848e-03,  5.56793734e-02,  4.25161161e-02,
         7.90285692e-03, -3.19814049e-02,  5.83508089e-02,
        -8.68604705e-02,  4.04716320e-02, -9.51927379e-02,
        -6.33105263e-02, -1.60308164e-

In [ ]:
# Retrieve the first three items by their IDs
specific_items = collection.get(
    ids=["doc_0", "doc_1", "doc_2"],
    include=["documents", "metadatas"] # We don't need the huge vectors
)

print("\n--- Specific Items ---")
pprint.pprint(specific_items)


--- Specific Items ---
{'data': None,
 'documents': ['Front Strut Coil Spring Sway Bar Link Kit - 4 Piece - '
               'Compatible with 2010-2016 Cadillac SRX (without RPO Code F45)\n'
               'Front Strut Coil Spring Sway Bar Link Kit 4 Piece Set - '
               'Compatible with 2010 - 2016 Cadillac SRXPosition  FrontNote  '
               'Front; without RPO Code F45  Real Time Damping Chassis ; '
               'Except RPO Code F45  Real Time Damping Chassis Quantity  4 '
               'Piece SetPackage Contents  Sway Bar LinksFeatures   - Matched '
               'Parts - Pre-AssembledCompatible With or Fits   - 2010 - 2016 '
               'Cadillac SRX Front; without RPO Code F45  Real Time Damping '
               'Chassis ; Except RPO Code F45  Real Time',
               'GASLAND Chef ES609MBN1 Single Wall Oven, 24" Built-in Electric '
               'Wall Oven, 240V 3200W 2.3Cu.f Convection Wall Oven with '
               'Rotisserie, 9 Cooking Modes, Mechani

##12 **Claude RAG Prediction Function**

In [ ]:
## 12. **Claude RAG Prediction Function**

def claude_rag(item):
    """
    Use Claude with RAG to predict item price
    """
    # Get similar items from vector DB
    documents, prices = find_similars(item)

    # Build multi-shot prompt with similar items as examples
    user_message = "You are a price estimation expert. Based on similar products, estimate the price of the given item.\n\n"
    user_message += "Here are some similar products with their prices:\n\n"

    # Add each similar item as context
    for i, (doc, price) in enumerate(zip(documents, prices), 1):
        user_message += f"Example {i}:\n{doc}\nPrice: ${price:.2f}\n\n"

    user_message += f"Now estimate the price for this item:\n{description(item)}\n\n"
    user_message += "Respond with ONLY the numeric price value (no dollar sign or other text)."

    # Call Claude API
    message = claude_client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=100,
        messages=[
            {"role": "user", "content": user_message}
        ]
    )

    # Extract and parse response
    reply = message.content[0].text.strip()

    return get_price(reply)

print("✓ Claude RAG function defined")

✓ Claude RAG function defined


In [4]:
## 13. **Test Claude RAG Pipeline**

print("="*70)
print("TESTING CLAUDE RAG PIPELINE")
print("="*70)

start_time = time.time()
Tester.test(claude_rag, test)
end_time = time.time()

total_time = end_time - start_time
print(f"\n⏱️ Processing time: {total_time:.2f} seconds")
print(f"⏱️ Time per item: {total_time / len(test):.2f} seconds")
print("="*70)

⚠️ Path not found: /content/drive/MyDrive/claude-LLM-PricePredictor
Attempting to mount Google Drive...
Mounted at /content/drive
TESTING CLAUDE RAG PIPELINE


NameError: name 'claude_rag' is not defined